In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
from extractor.forex_extractor import FOREXExtractor
from time import sleep

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [4]:
market.connect()
start = market.retrieve_max_date("stocks")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2023-05-27 2023-06-07


In [5]:
market.connect()
data = FOREXExtractor.extract(start,end)
values = pd.DataFrame(data["rates"].values.tolist())
dates = data["rates"].keys().tolist()
values["dates"]= dates
market.store("forex",values)
market.disconnect()

In [6]:
values

,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,dates
0,3.672204,87.000368,103.203989,386.520403,1.802099,552.503981,235.614184,1.534204,1.8025,1.703970,...,2.70255,0.746965,610.503597,111.650364,250.350363,19.659890,9001.203589,19.273756,321.999592,2023-05-27
1,3.672196,87.751525,103.174011,386.121627,1.805662,552.499375,236.199189,1.532180,1.8025,1.703975,...,2.70255,0.748466,611.637730,111.649636,250.349650,19.656825,9001.202122,19.311676,321.999592,2023-05-28
2,3.672698,87.567063,103.057673,386.170070,1.801819,567.144410,238.238898,1.531816,1.8025,1.692332,...,2.70255,0.750828,612.037210,111.650241,250.350189,19.664865,9001.205074,19.346291,321.999592,2023-05-29
3,3.672901,88.000126,102.669691,386.520179,1.801937,564.999763,238.838854,1.534295,1.8000,1.698524,...,2.70255,0.750795,613.559540,111.824969,250.297717,19.687770,9001.199565,19.372691,321.999592,2023-05-30
4,3.672898,87.499662,102.849752,386.620039,1.801170,583.500036,239.471624,1.537019,1.8025,1.699477,...,2.70255,0.750707,614.501055,112.197509,250.350073,19.729950,9001.200609,19.491986,321.999592,2023-05-31
5,3.673103,87.164720,102.089488,386.980286,1.801558,589.521970,240.187723,1.520300,1.8025,1.696786,...,2.70255,0.750673,612.297152,112.202905,250.350249,19.625897,9001.182634,19.517135,321.999592,2023-06-01
6,3.672904,87.503991,101.450403,387.220403,1.802559,596.503981,240.409640,1.510350,1.8025,1.703970,...,2.70255,0.751093,609.503597,111.750364,250.350363,19.510070,9001.203589,19.712880,321.999592,2023-06-02
7,3.672904,87.512236,101.477649,387.078730,1.809071,596.503981,241.112478,1.512402,1.8025,1.703970,...,2.70255,0.753834,611.529390,111.750364,250.350363,19.510380,9001.203589,19.784646,321.999592,2023-06-03
8,3.672935,87.512236,101.477649,387.078730,1.809071,596.500185,241.762724,1.515611,1.8025,1.697294,...,2.70255,0.753834,611.529390,111.750171,250.350180,19.550978,9001.197558,19.784646,321.999592,2023-06-04
9,3.672901,87.495940,100.602368,384.150411,1.799002,606.000224,242.469366,1.512140,1.8000,1.700499,...,2.70255,0.748223,612.354311,111.702991,250.305638,19.276201,9001.203933,19.815068,321.999592,2023-06-05


In [7]:
market.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        try:
            data = TiingoExtractor.extract(ticker,start,end)
            data["ticker"] = ticker
            market.store("stocks",data)
        except:
            print(ticker,"tiingo")
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

  1%|█▏                                                                                                                       | 5/490 [00:03<04:59,  1.62it/s]

market stocks documents must be a non-empty list


 73%|██████████████████████████████████████████████████████████████████████████████████████▉                                | 358/490 [03:35<01:18,  1.68it/s]

market stocks documents must be a non-empty list


 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████            | 441/490 [04:25<00:30,  1.63it/s]

market stocks documents must be a non-empty list


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [04:54<00:00,  1.66it/s]


In [9]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:32<00:00, 14.93it/s]


In [10]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
75,0.479008,0.265008,0.273971,0.198063,0.115162,AVGO,Broadcom Inc.,Information Technology,Semiconductors
47,0.408899,0.015560,0.063624,0.171837,0.090589,ANET,Arista Networks,Information Technology,Communications Equipment
144,-0.027216,0.059687,0.020221,0.080647,0.072327,DLR,Digital Realty,Real Estate,Specialized REITs
312,0.521990,0.056697,0.104252,0.161731,0.066760,MPWR,Monolithic Power Systems,Information Technology,Semiconductors
428,0.208162,-0.028933,0.123619,0.061542,0.066207,TER,Teradyne,Information Technology,Semiconductor Equipment
196,0.060647,-0.034379,0.002488,0.039553,0.062390,F,Ford Motor Company,Consumer Discretionary,Automobile Manufacturers
304,0.470621,0.242664,0.185726,0.116262,0.062060,MU,Micron Technology,Information Technology,Semiconductors
265,0.228802,0.155894,0.199672,0.095958,0.062038,KLAC,KLA Corporation,Information Technology,Semiconductor Equipment
377,0.122280,-0.008228,0.077553,0.040782,0.060982,QRVO,Qorvo,Information Technology,Semiconductors
379,0.035627,-0.115360,-0.047393,0.048157,0.060854,QCOM,Qualcomm,Information Technology,Semiconductors


In [11]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Information Technology,0.159531,0.015108,0.052227,0.022080,0.027426
Consumer Staples,0.002039,-0.018336,-0.056061,-0.015293,0.000735
Industrials,0.037184,-0.019739,-0.017349,-0.013596,0.007779
Consumer Discretionary,0.064245,-0.027270,-0.036998,-0.020269,0.007701
Communication Services,0.064514,-0.027546,-0.015200,-0.014655,0.018119
Financials,-0.095129,-0.033449,-0.032206,-0.014583,0.006844
Health Care,-0.029710,-0.037309,-0.063662,-0.035493,-0.000327
Real Estate,-0.057706,-0.042988,-0.057422,-0.024428,0.010003
Utilities,-0.068878,-0.045495,-0.069908,-0.026302,-0.001547
